# Transolver
## 1. 模型简介
这是一个用于汽车设计任务的模型，它用于预测驾驶汽车周围的风速和表面压力，以此来计算风阻系数。模型参照[Transolver](https://arxiv.org/abs/2402.02366) 中的开源模型代码进行构建。

**模型结构**<br>
主要基于Transformer架构,用Physics-Attention代替了标准的Attention。每个头将输入域编码为一系列物理感知令牌，然后通过令牌之间的注意力来捕获复杂几何形状下的物理相关性。


## 2. 软件环境准备

### 2.1 基于dtk适配软件
<div class="alert alert-warning"> WARNING：镜像中环境已配置，此步骤省略 </div>

### 2.2 软件环境检查


In [1]:
# 检查torch版本
import torch
import os
import onescience

version = torch.__version__
num = float(version[:3])
# assert num == 1.10

# 检查硬件环境
device = "cpu"
if os.system('rocm-smi 2>/dev/null || hy-smi 2>/dev/null')==0:
    device = "dtk"

elif os.system('nvidia-smi 2>/dev/null')==0: 
    device = "cuda"

print("torch version:", version)
print("device =", device)
print("onescience version:", onescience.__version__)


============================ System Management Interface =============================
HCU     Temp     AvgPwr     Perf     PwrCap     VRAM%      HCU%      Mode     
0       48.0C    30.0W      auto     300.0W     0%         0%        N/A      
1       49.0C    23.0W      auto     300.0W     0%         0%        N/A      
2       50.0C    24.0W      auto     300.0W     0%         0%        N/A      
3       50.0C    23.0W      auto     300.0W     0%         0%        N/A      
=================================== End of SMI Log ===================================

torch version: 2.4.1
device = dtk
onescience version: 0.1.0


### 2.2 软件依赖安装


<div class="alert alert-warning"> WARNING: 镜像中环境已配置，此步骤省略 </div>

<div class="alert alert-note" style="color: blue;">
Note: 存在部分文件更新，请卸载当前环境下的 onescience 包，并安装当前目录的 whl 包。
    
请在最顶层目录(包含setup.py文件的目录)，通过命令行，执行下述安装指令：
</div>

In [ ]:
!pip install -e .

## 3. 素材准备
### 3.1 数据集准备

原始数据可以此处[[Shape-Net Car]](http://www.nobuyuki-umetani.com/publication/mlcfd_data.zip)，由[Nobuyuki Umetani](https://dl.acm.org/doi/abs/10.1145/3197517.3201325)提供


下载的数据解压后的目录如下所示，可以通过参数指定其中一个param文件夹下的所有数据用作验证或预测。本项目中只用到了vtk格式的数据，从其中提取网格、压力和速度信息存入preprocessed_data文件夹中用作后续训练或者推理。

```
mlcfd_data
│
├── graph_raw_data # 原始图表数据，用于展示数据集中的数据排布形式
│
├── linear_regression_code #线性回归相关的代码文件，用于基准模型或辅助分析
│
├── preprocessed_data #对原始数据经过预处理得到的数据集，供模型训练使用
│
├── side_by_side_comparisons  #对应param7中的数据模型结果与实际结果的对比
│
└── training_data #原始的训练数据
    └── param0  
        └── 1a0bc9ab92c915167ae33d942430658c
            ├── cd.txt  # 风阻系数 
            ├── hexvelo_smpl.vtk  # VTK格式的六面体网格速度采样数据，用于三维流场可视化或分析
            ├── param1.txt  # 参数1的文本文件，存储模仿真过程中的辅助参数
            ├── param2.txt  # 参数2的文本文件，存储模仿真过程中的辅助参数
            ├── press.npy  # 压力场数据的numpy数组文件，存储仿真得到的压力值
            ├── quadpress_smpl.vtk # VTK格式四面体网格压力采样数据，用于网格分析与可视化
            └── velo.npy # 速度场数据的numpy数组文件，存储流场速度信息           
        ├── 1a56d596c77ad5936fa87a658faf1d26
        ├── 1ab80bc91a45b7d0a31091d2234b9f68
        └── ...
   └── param1
   └── param2
   └── param3
   └── param4
   └── param5
   └── param6
   └── param7

```

## 4 训练


### 4.1 单卡训练
运行`python main.py`进行训练

主要参数介绍，可以修改参数更改训练模型的信息：
- data_dir：训练数据所在目录路径，包含所有原始CFD数据文件。默认`./dataset/mlcfd_data/training_data`
- preprocessed_save_dir：预处理数据保存目录，用于存储处理后的训练/验证数据。默认./dataset/mlcfd_data/preprocessed_data
- model_save_dir：训练后的模型保存目录。默认`./metrics`
- fold_id：选择哪个param子文件夹作为测试/验证集。默认`0`
- gpu：指定使用的GPU编号，默认为0，若无GPU则使用CPU。默认`0`
- val_iter：验证时每隔多少个训练迭代执行一次模型验证。默认`10`
- cfd_config_dir：CFD仿真参数配置文件路径。默认`cfd/cfd_params.yaml`
- cfd_model：选择的CFD模型名称。默认`Transolver`
- cfd_mesh：是否使用CFD网格信息，启用后会使用网格相关特征。默认`False`
- r：影响图神经网络邻域范围的半径参数。默认`0.2`
- weight：压力项损失的权重比例。默认`0.5`
- lr：模型训练时初始的学习率。默认`0.001`
- batch_size：训练或验证时的批大小。默认`1`
- nb_epochs：总训练轮数。默认`200`
- preprocessed：是否使用预处理数据，1为使用，0为不使用。默认`1`

In [ ]:
!python main.py --cfd_model=Transolver --data_dir=./dataset/mlcfd_data/training_data --preprocessed_save_dir=./dataset/mlcfd_data/preprocessed_data --gpu=0 

### 4.2 多卡训练
运行`mpirun -np <num_GPUs> --allow-run-as-root python main.py`进行训练

主要参数介绍，其他参数参见单卡介绍：
- num_GPUs：卡的数量

In [ ]:
!mpirun -np 4 --allow-run-as-root python main.py

### 4.3 跨节点多卡训练

torchrun启动多节点多卡训练：`torchrun --standalone --nnodes=<num_nodes> --nproc_per_node=<num_GPUs> main.py`
主要参数介绍，其他详细可参考torchrun参数：
- num_nodes： 节点数量
- num_GPUs：卡的数量

In [ ]:
!torchrun --standalone --nnodes=1 --nproc_per_node=4 main.py

## 5 推理

### 5.1 推理
运行`python main_evaluation.py`进行推理

主要参数介绍，其余均与训练时的参数一致，可以通过`python main_evaluation.py -h`查看：
- result_dir：推理结果保存的根目录，包括npy、vtk、vis等子目录。默认`./results`
- save_vtk：是否保存推理预测结果为VTK文件，便于后续可视化。。默认`False`
- visualize：是否基于VTK文件生成可视化图片，需配合`--save_vtk`使用。。默认`False`

In [ ]:
!python main_evaluation.py --result_dir=./results